In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

import os

Using TensorFlow backend.


In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# 데이터 불러오기
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [0]:
batch_size = 16
num_classes=10
epochs=25

In [6]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [0]:
# 벡터화
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
print('Using real-time data augmentation.')
print('Using real-time data augmentation.')


Using real-time data augmentation.
Using real-time data augmentation.


In [11]:
print('Using real-time data augmentation.')

datagen = ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,  
    zca_whitening=False, 
    zca_epsilon=1e-06,
    rotation_range=0, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0., 
    zoom_range=0., 
    channel_shift_range=0.,  
    fill_mode='nearest',
    cval=0., 
    horizontal_flip=True, 
    vertical_flip=False, 
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0)

datagen.fit(x_train)

Using real-time data augmentation.


In [0]:
filepath = "./saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [13]:
# for example
nkernels = 32
model = Sequential()

# b, 32, 32, 3
model.add(Conv2D(nkernels, 3, activation='relu', kernel_initializer='he_normal', padding='valid', input_shape=(32,32,3)))
model.add(Conv2D(nkernels, 3, activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# b, 16, 16, 64
model.add(Conv2D(nkernels*2, 3, activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(Conv2D(nkernels*4, 3, activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# b, 8, 8, 128
model.add(Conv2D(nkernels*8, 3, activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(BatchNormalization())

model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation='softmax', kernel_initializer='glorot_uniform'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 10, 128)      

In [0]:
opt = keras.optimizers.rmsprop(lr= 0.0001, decay=1e-6)

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [17]:
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs, validation_data=(x_test, y_test), workers=4, callbacks=[checkpoint])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
3125/3125 [==============================] - 47s 15ms/step - loss: 1.4913 - acc: 0.4690 - val_loss: 1.2239 - val_acc: 0.5646

Epoch 00001: saving model to ./saved-model-01-0.56.hdf5
Epoch 2/25
3125/3125 [==============================] - 39s 13ms/step - loss: 1.2257 - acc: 0.5729 - val_loss: 1.1788 - val_acc: 0.5895

Epoch 00002: saving model to ./saved-model-02-0.59.hdf5
Epoch 3/25
3125/3125 [==============================] - 38s 12ms/step - loss: 1.1194 - acc: 0.6117 - val_loss: 1.0486 - val_acc: 0.6330

Epoch 00003: saving model to ./saved-model-03-0.63.hdf5
Epoch 4/25
3125/3125 [==============================] - 39s 12ms/step - loss: 1.0464 - acc: 0.6358 - val_loss: 1.1017 - val_acc: 0.6358

Epoch 00004: saving model to ./saved-model-04-0.64.hdf5
Epoch 5/25
3125/3125 [==============================] - 38s 12ms/step - loss: 0.9871 - acc: 0.6566 - val_loss: 0.8896 - val_acc: 0.6